## FEC 2018 Campaign Finance: 

### Operating expenditures

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from copy import deepcopy
from datetime import datetime as dt

%matplotlib inline

In [19]:
# add column headers from separate file
headers = pd.read_csv('assets/oppexp_header_file.csv')

# read in & clean
opp = pd.read_csv('assets/oppexp18.zip', sep = '|', error_bad_lines = False, header = None)
print('original length:', len(opp))

opp.dropna(axis = 1, how = 'all', inplace = True)

opp.columns = [x.lower() for x in headers.columns]

opp.head(2)

,cmte_id,amndt_ind,rpt_yr,rpt_tp,image_num,line_num,form_tp_cd,sched_tp_cd,name,city,...,purpose,category,category_desc,memo_cd,memo_text,entity_tp,sub_id,file_num,tran_id,back_ref_tran_id
0,C00626739,N,2017,YE,201801249090614233,21B,F3X,SB,ACTBLUE TECHNICAL SERVICES,SOMERVILLE,...,ONLINE CREDIT CARD PROCESSING FEE,003,Solicitation and Fundraising Expenses,NaN,NaN,ORG,4012420181503169942,1199698,VTDEGA3ZWX3,NaN
1,C00626739,N,2017,YE,201801249090614238,21B,F3X,SB,FACEBOOK,MENLO PARK,...,ANTI-PAUL RYAN ADVERTISING,004,Advertising Expenses,NaN,NaN,ORG,4012420181503169972,1199698,VTDEGA3ZFM4,NaN


In [22]:
# inspect only lines from opp corresponding to cand_id in df
cm = pd.read_csv('assets/committees_2018.csv')
print(len(cm))
cms = list(set(cm['cmte_id']))
mask = [True if opp.loc[i, 'cmte_id'] in cms else \
        False for i in list(opp.index)]
opp = opp[mask]
print(len(opp))

opp.head(2)

,cmte_id,amndt_ind,rpt_yr,rpt_tp,image_num,line_num,form_tp_cd,sched_tp_cd,name,city,...,purpose,category,category_desc,memo_cd,memo_text,entity_tp,sub_id,file_num,tran_id,back_ref_tran_id
0,C00626739,N,2017,YE,201801249090614233,21B,F3X,SB,ACTBLUE TECHNICAL SERVICES,SOMERVILLE,...,ONLINE CREDIT CARD PROCESSING FEE,003,Solicitation and Fundraising Expenses,NaN,NaN,ORG,4012420181503169942,1199698,VTDEGA3ZWX3,NaN
1,C00626739,N,2017,YE,201801249090614238,21B,F3X,SB,FACEBOOK,MENLO PARK,...,ANTI-PAUL RYAN ADVERTISING,004,Advertising Expenses,NaN,NaN,ORG,4012420181503169972,1199698,VTDEGA3ZFM4,NaN


In [23]:
opp['amndt_ind'].value_counts()

N    1094558
A     695493
T       9022
Name: amndt_ind, dtype: int64

In [24]:
opp['rpt_yr'].value_counts()

2018    1171475
2017     627598
Name: rpt_yr, dtype: int64

In [25]:
opp['rpt_tp'].value_counts()

Q3     310190
Q2     274323
YE     266490
Q1     233908
30G    147498
12P     92213
12G     68240
M9      40568
M10     39859
M8      38189
M7      35340
M6      33938
MY      33936
M5      32235
M4      31241
M3      25586
M2      22976
M12     14368
M11     14162
12S     12631
TER     11235
30S      8137
12R      7889
30R      2449
12C      1448
30P        22
 M4         2
Name: rpt_tp, dtype: int64

In [26]:
opp['name'].value_counts()

ACTBLUE TECHNICAL SERVICES              56589
UBER                                    27046
PAYPAL FEE                              21898
ANEDOT                                  21463
AMERICAN AIRLINES                       21338
                                        ...  
1000 UNIVERSITY AVENUE PROPERTIES,LP        1
CHRISTIAN NAKKASHIAN                        1
MULHOLLAND, BOB                             1
LEISHMAN, WADE                              1
MRHAB INVESTMENT LLC                        1
Name: name, Length: 171264, dtype: int64

In [29]:
opp['category_desc'].value_counts()

Administrative/Salary/Overhead Expenses     312796
Travel Expenses                              73318
Solicitation and Fundraising Expenses        58663
Advertising Expenses                         29832
Campaign Event Expenses                      13233
Campaign Materials                           11136
Political Contributions                       4228
Polling Expenses                              2409
Donations                                     1410
Transfers                                      170
Loan Repayments                                137
Refunds of Contributions                       129
Name: category_desc, dtype: int64

In [28]:
opp.loc[opp['name'] == 'UBER', 'purpose'].value_counts()

TRAVEL                                      12523
TRANSPORTATION                               2820
TRAVEL/ACCOMMODATION /MEALS                  2352
TRAVEL EXPENSE                                901
RIDE SHARE SERVICE                            677
                                            ...  
PAC CAB FARE                                    1
CAREFARE                                        1
UBER CAB FARE (DUSTY JOHNSON)                   1
CAR SERVICE - TYNGSBORO TO BOSTON               1
REIMBURSEMENT [SB17.12955]: TRAVEL:GROUN        1
Name: purpose, Length: 555, dtype: int64

In [ ]:
cmtetp_dict = {
    'H' : 'cmte_House',
    'S' : 'cmte_Senate',
    'N' : 'cmte_PACnonqual',
    'Q' : 'cmte_PACqual',
    'Y' : 'cmte_Partyqual',
    'O' : 'cmte_SuperPAC',
    'U' : 'cmte_Singlecand',
}

opp['cmte_tp'] = [cmtetp_dict[x] for x in opp['cmte_tp']]
opp['cmte_tp'].value_counts()

In [ ]:
cmtedsgn_dict = {
    'A' : 'Authorized by a candidate',
    'B' : 'Lobbyist/Registrant PAC',
    'D' : 'Leadership PAC',
    'J' : 'Joint fundraiser',
    'P' : 'Principal campaign committee',
    'U' : 'Unauthorized',
}

opp['cmte_dsgn'] = [cmtedsgn_dict[x] for x in opp['cmte_dsgn']]
opp['cmte_dsgn'].value_counts()

In [ ]:
set(opp['linkage_id'].value_counts())

In [ ]:
opp.head()

In [ ]:
opp.to_csv('assets/cand_comm_linkages.csv', index = False)